In [33]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [46]:
original_df = pd.read_csv("CountryTemperatures_byYear.csv").drop(columns=["Unnamed: 0"])
original_df.head()

,year,Country,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,GHG,temp_diff
0,1900,AFGHANISTAN,13.749333,0.748833,33.00,65.0,0.0,0.0
1,1900,ALBANIA,13.068583,0.723417,41.00,20.0,0.0,0.0
2,1900,ALGERIA,22.864167,0.826667,28.00,3.0,0.0,0.0
3,1900,AMERICAN SAMOA,26.273500,0.891000,-14.33,-170.0,NaN,0.0
4,1900,ANDORRA,11.348333,0.542167,42.50,1.5,0.0,0.0


In [47]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24342 entries, 0 to 24341
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   year                           24342 non-null  int64  
 1   Country                        24342 non-null  object 
 2   AverageTemperature             24341 non-null  float64
 3   AverageTemperatureUncertainty  24341 non-null  float64
 4   Latitude                       24342 non-null  float64
 5   Longitude                      24342 non-null  float64
 6   GHG                            22458 non-null  float64
 7   temp_diff                      24341 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 1.5+ MB


In [51]:
def get_temperature_diff_between(df_by_year,start_date,end_date):
    df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
                        (df_by_year["year"] <= end_date)].groupby(["Country"]).sum().reset_index()
    
    return df

In [52]:
df = get_temperature_diff_between(original_df,1950,2000)
df.head()

,Country,year,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,GHG,temp_diff
0,AFGHANISTAN,100725,728.755833,21.659750,1683.00,3315.0,1.438653e+09,2.147750
1,ALBANIA,100725,658.302667,17.186667,2091.00,1020.0,4.387325e+09,1.228333
2,ALGERIA,100725,1188.247167,19.266083,1428.00,153.0,3.047590e+10,0.844083
3,AMERICAN SAMOA,100725,1369.174667,19.267583,-730.83,-8670.0,0.000000e+00,0.368833
4,ANDORRA,100725,588.063083,16.091250,2167.50,76.5,2.803693e+07,-0.006000


In [61]:
fig = px.scatter(df, x="temp_diff", y="GHG", color="Country", trendline="ols", template="simple_white",
                labels={
                     "temp_diff": "Average Temperature evolution",
                     "GHG": "Greenhouse Gases Emissions"

                 },)
fig.show()

In [ ]:
import plotly.graph_objs as go
import statsmodels.api as sm

fig = px.scatter(df, x="temp_diff", y ="GHG", size=np.log(df["GHG"]), color="temp_diff", hover_name=df.index, log_x=False,marginal_x = "box",marginal_y = "box", template="simple_white", color_discrete_sequence=["#0d0887", "#9c179e"])

# linear regression
regline = sm.OLS(df["AverageTemperature"],sm.add_constant(np.log(df["GHG"]))).fit().fittedvalues

# add linear regression line for whole sample
fig.add_traces(go.Scatter(x=np.log(df["GHG"]), y=regline,
                          mode = 'lines',
                          marker_color='#fb9f3a',
                          name='OLS Trendline')
                          )

fig.show()

In [45]:
df = df.dropna()
df= df[df['GHG'] != 0]
df

KeyError: 'GHG'

In [23]:
df = df.loc[(df['year'] == 1949)].groupby('Country')[['AverageTemperature', 'GHG']].mean().reset_index()

In [56]:
fig = px.scatter(df, x="AverageTemperature", y="GHG", color="Country", marginal_y="violin",
           marginal_x="box", trendline="ols", template="simple_white")
fig.show()

In [25]:
import plotly.graph_objs as go
import statsmodels.api as sm

col_name = str("AverageTemperature") + "(above Average)"
df[col_name] = (df[("AverageTemperature")] > df[("AverageTemperature")].mean())#.astype(int)

fig = px.scatter(df, x="AverageTemperature", y ="GHG", size=np.log(df["GHG"]), color=col_name, hover_name=df.index, log_x=False,marginal_x = "box",marginal_y = "box", template="simple_white", color_discrete_sequence=["#0d0887", "#9c179e"])

# linear regression
regline = sm.OLS(df["AverageTemperature"],sm.add_constant(np.log(df["GHG"]))).fit().fittedvalues

# add linear regression line for whole sample
fig.add_traces(go.Scatter(x=np.log(df["GHG"]), y=regline,
                          mode = 'lines',
                          marker_color='#fb9f3a',
                          name='OLS Trendline')
                          )

fig.show()

In [26]:
col_name = str("AverageTemperature") + ": above Average"
df[col_name] = (df[("AverageTemperature")] > df[("AverageTemperature")].mean())#.astype(int)

    
size = df["AverageTemperature"]
    
fig = px.scatter(df, x="AverageTemperature", y ="GHG", size=size, color=col_name,hover_name=df.index, log_x=False, trendline = "ols",  trendline_color_override="#bd3786",marginal_x = "box",marginal_y = "box", template="simple_white", color_discrete_sequence=px.colors.qualitative.G10)
fig.show()

ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [-4.948583333333334]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [27]:
def scatter_plot(df, x, y, size):
    '''insert x and y as string while x entails color coding (abv avg...)'''
    
    # feature engineering
    col_name = str(y) + " above avg"
    df[col_name] = (df[y] > df[y].mean()).astype(int)
    
    size = df[size] * 1/4
    
    # plotting
    fig = px.scatter(df, x=x, y =y, size=size, color_continuous_scale=px.colors.sequential.Plasma, color=col_name,hover_name=df.index, log_x=False, trendline = "ols",  trendline_color_override="#bd3786",marginal_x = "box",marginal_y = "box", template="simple_white")
    fig.show()

In [28]:
scatter_plot(df , "AverageTemperature", "GHG", "AverageTemperature")

ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [-1.2371458333333334]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [29]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df["Country"],
    y=np.log(df["GHG"]),
    name="GHG EMission",
    marker_color='#0d0887'
))
fig.add_trace(go.Bar(
    x=df["Country"],
    y=df["AverageTemperature"],
    name="Average Temperature",
    marker_color='#fdca26'
))

In [30]:
def get_temperature_diff_between(df_by_year,start_date,end_date):
    df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
                        (df_by_year["year"] <= end_date)].groupby(["Country"]).sum()[["temp_diff"]]
    
    return df